# 预处理

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import random
import numpy as np
import pandas as pd
import lightkurve as lk
import tensorflow as tf
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline

In [3]:
def lc_split(filename,gap_width):
    table = fits.getdata(filename)
    time = table['TIME']
    flux = table['PDCSAP_FLUX']
    q_lst = table['QUALITY']
    m = q_lst == 0
    time = time[m]
    flux = flux[m]
    m2 = ~np.isnan(flux)
    all_time = time[m2]
    all_flux = flux[m2]
    
    # Handle single-segment inputs.
    if isinstance(all_time, np.ndarray) and all_time.ndim == 1:
        all_time = [all_time]
        all_flux = [all_flux]

    out_time = []
    out_flux = []
    for time, flux in zip(all_time, all_flux):
        start = 0
        for end in range(1, len(time) + 1):
            # Choose the largest endpoint such that time[start:end] has no gaps.
            if end == len(time) or time[end] - time[end - 1] > gap_width:
                out_time.append(time[start:end])
                out_flux.append(flux[start:end])
                start = end

    return out_time, out_flux

In [10]:
def robust_mean(y, cut):
    """Computes a robust mean estimate in the presence of outliers.
    Args:
        y: 1D numpy array. Assumed to be normally distributed with outliers.
        cut: Points more than this number of standard deviations from the median areignored.
    Returns:
        mean: A robust estimate of the mean of y.
        mean_stddev: The standard deviation of the mean.
        mask: Boolean array with the same length as y. Values corresponding tooutliers in y are False. All other values are True.
    """
    # First, make a robust estimate of the standard deviation of y, assuming y is
    # normally distributed. The conversion factor of 1.4826 takes the median
    # absolute deviation to the standard deviation of a normal distribution.
    # See, e.g. https://www.mathworks.com/help/stats/mad.html.
    absdev = np.abs(y - np.median(y))
    sigma = 1.4826 * np.median(absdev)

    # If the previous estimate of the standard deviation using the median absolute
    # deviation is zero, fall back to a robust estimate using the mean absolute
    # deviation. This estimator has a different conversion factor of 1.253.
    # See, e.g. https://www.mathworks.com/help/stats/mad.html.
    if sigma < 1.0e-24:
        sigma = 1.253 * np.mean(absdev)

    # Identify outliers using our estimate of the standard deviation of y.
    mask = absdev <= cut * sigma

    # Now, recompute the standard deviation, using the sample standard deviation
    # of non-outlier points.
    sigma = np.std(y[mask])

    # Compensate the estimate of sigma due to trimming away outliers. The
    # following formula is an approximation, see
    # http://w.astro.berkeley.edu/~johnjohn/idlprocs/robust_mean.pro.
    sc = np.max([cut, 1.0])
    if sc <= 4.5:
        sigma /= (-0.15405 + 0.90723 * sc - 0.23584 * sc**2 + 0.020142 * sc**3)

    # Identify outliers using our second estimate of the standard deviation of y.
    mask = absdev <= cut * sigma

    # Now, recompute the standard deviation, using the sample standard deviation
    # with non-outlier points.
    sigma = np.std(y[mask])

    # Compensate the estimate of sigma due to trimming away outliers.
    sc = np.max([cut, 1.0])
    if sc <= 4.5:
        sigma /= (-0.15405 + 0.90723 * sc - 0.23584 * sc**2 + 0.020142 * sc**3)

    # Final estimate is the sample mean with outliers removed.
    mean = np.mean(y[mask])
    mean_stddev = sigma / np.sqrt(len(y) - 1.0)

    return mean, mean_stddev, mask

In [4]:
def process_lc(time,flux,outlier_cut,flux_len):
    '''
    time 一维时间序列
    flux 一维光变曲线
    outlier_cut 与中位数的最大标准差数点之前的样条残差被认为是离群值。
    
    '''
    # 将时间归一化至[0, 1].
    t_min = np.min(time)
    t_max = np.max(time)
    time = (time - t_min) / (t_max - t_min)

    #mask是一个bool型向量，其中TRUE表示用来参与插值拟合的数据
    mask = np.ones_like(time, dtype=np.bool_)
    
    #迭代地拟合样条
    maxiter=3#迭代次数
    for _ in range(maxiter):
        time = time[mask]
        flux = flux[mask]
        spl = InterpolatedUnivariateSpline(time, flux)
        fit = spl(time)
        residuals = flux - fit
        new_mask = robust_mean(residuals, cut=outlier_cut)[2]
        # if np.all(new_mask == mask):
        #     break  # Spline converged.
        mask = new_mask
    
    #利用拟合好的样条产生模拟数据
    x = np.linspace(0, 1, flux_len)
    fit_flux = spl(x)
    
    #归一化光变曲线
    f_max = max(fit_flux)
    f_min = min(fit_flux)
    f = (fit_flux - f_min)/(f_max - f_min)
    
    return f 

In [5]:
#按比例取部分数据，ratio：0-100，保留数据的比例
def dropMax(t,f,ratio):
    mask = f < np.percentile(f, ratio)
    f = f[mask]
    t = t[mask]
    return t,f

In [6]:
import csv
def writeData(data,csv_file):
    # 如果文件不存在，新建一个
    if not os.path.exists(csv_file):
        with open(csv_file, 'w', newline='') as f:
            writer = csv.writer(f)

    # 写入行数据
    with open(csv_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(data)

# lc

In [8]:
#将划分好的lc数据存入文件中
gap_width = 0.75 #linagyu代码中引用
outlier_cut = 3
flux_len = 4000
labelLst = ['EA','EB','EW']

    
def getProcessedData(label,gap_width,outlier_cut,flux_len):
    dir = "./labeledLcData/"+label+'/'
    #将划分好的lc数据存入文件中
    csv_file = './processedData_4000/'+label+'.csv'
    fileLst = os.listdir(dir)
    for file in fileLst:
        filename = dir + file
        out_time,out_flux = lc_split(filename,gap_width)
        for t,f in zip(out_time,out_flux):
            if len(t) >= 2000:
                t,f = dropMax(t,f,99)
                final_flux = process_lc(t,f,outlier_cut,flux_len)
                writeData(final_flux,csv_file)
                
for label in labelLst:
    getProcessedData(label,gap_width,outlier_cut,flux_len)

# GLS

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import periodogram


def getGLSData(label,gap_width,period_lst):
    dir = "./labeledLcData/"+label+'/'
    #将划分好的lc数据存入文件中
    csv_file = './GLSdata_1000/'+ 'new_' + label+'.csv'
    fileLst = os.listdir(dir)
    for file in fileLst:
        filename = dir + file
        out_time,out_flux = lc_split(filename,gap_width)
        for t,f in zip(out_time,out_flux):
            t,f = dropMax(t,f,99)
            if len(t) >= 2000:
                t,f = dropMax(t,f,99)
                pdm = periodogram.GLS(t, f)
                power, winpower = pdm.get_power(period=period_lst)
                #power归一化
                p_max = max(power)
                p_min = min(power)
                p = (power - p_min)/(p_max - p_min)
                writeData(p,csv_file)

In [12]:
labelLst = ['EA','EB','EW']
gap_width = 0.75
period_lst = np.logspace(-3, 1, 1125)[125:]

for label in labelLst:
    getGLSData(label,gap_width,period_lst)

# all TESS data

In [7]:
#将扇区数据进行lc预处理和获得GLS周期提数据，并保存
def getProcessedData(sec,gap_width,outlier_cut,flux_len,period_lst):
    #存入文件位置
    lc_csv_file = 'F:/tess/processedData/lc/'+sec+'.csv'
    GLS_csv_file = 'F:/tess/processedData/GLS/'+sec+'.csv'
    
    dir = "F:/tess/lc/"+sec+'/'
    fileLst = os.listdir(dir)
    for file in fileLst:
        filename = dir + file
        out_time,out_flux = lc_split(filename,gap_width)
        for t,f in zip(out_time,out_flux):
            if len(t) >= 2000:
                t,f = dropMax(t,f,99)#去除离群
                #lc预处理并保存,保存时第一列保存文件名
                final_flux = process_lc(t,f,outlier_cut,flux_len)
                save_flux = [file]+list(final_flux)
                writeData(save_flux,lc_csv_file)
                
                #获取power，GLS数据，并保存
                pdm = periodogram.GLS(t, f)
                power, winpower = pdm.get_power(period=period_lst)
                #power归一化
                p_max = max(power)
                p_min = min(power)
                p = (power - p_min)/(p_max - p_min)
                save_power = [file]+list(p)
                writeData(save_power,GLS_csv_file)

In [ ]:
import periodogram
#将划分好的lc数据存入文件中
gap_width = 0.75 #linagyu代码中引用
outlier_cut = 3
flux_len = 4000
period_lst = np.logspace(-3, 1, 1125)[125:]

sectorlst = ['s075']

for sec in sectorlst:
    getProcessedData(sec,gap_width,outlier_cut,flux_len,period_lst)